# Llama 3.2 한글 Test

Test Model

In [1]:
%pip install transformers==4.46.3
%pip install datasets==3.2.0
%pip install accelerate==1.2.1
%pip install peft==0.14.0
%pip install trl==0.12.2
%pip install bitsandbytes==0.45.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# The model that you want to train from the Hugging Face hub
base_model = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# Base Model

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer
import torch

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import torch
res = torch.cuda.is_available()
print(res)

True


In [5]:

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Set pad_token_id to avoid receiving warning messages.

In [6]:
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if model.config.pad_token_id is None:
    model.config.pad_token_id = model.config.eos_token_id

In [7]:
def generate_response(messages, model):

    input_ids = tokenizer.apply_chat_template(
         messages, add_generation_prompt=True,
         return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    decoded_output = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

    return decoded_output

instruction = "BTS 음악의 컨셉은 무엇입니까?"
messages = [
    {"role": "user", "content": f"{instruction}"},
]

generate_response(messages, model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'BTS(방탄소년단)는 K-POP 그룹으로, 다양한 음악 컨셉을 통해 다양한 주제와 메시지를 전달하고 있습니다. 그들의 음악 컨셉은 주로 개인의 성장, 사랑, 사회적 문제, 그리고 현대 사회에 대한 비판 등을 다루고 있습니다.\n\n1. **성장과 자아 발전**: BTS는 자신의 음악을 통해 개인의 성장과 자아 발전을 다루고 있습니다. 예를 들어, "DNA", "Mic Drop", "Boy With Luv" 등의 곡은 자신의 강점을 강조하고, 자신의 성장을 위해 노력하는 메시지를 전달합니다.\n\n2. **사랑과 관계**: 사랑과 관계에 대한 다양한 주제를 다루고 있습니다. "Love Myself", "Spring Day", "Euphoria" 등의 곡은 사랑과 관계를 통해 다양한 감정을 표현하고 있습니다.\n\n3. **사회적 문제와 사회적 비판**: BTS는 사회적 문제에 대해 비판적이고 반성적인 메시지를 전달하고 있습니다. "Not Today", "War of Hormone", "Spring Day" 등의 곡은 사회적 문제를 다루고 있으며, 이를 해결하기 위한 방법을 제안합니다.\n\n4. **현대 사회와 문화**: 현대 사회와 문화에 대한 비판적이고 반성적인 메시지를 전달하고 있습니다. "Fake Love", "Idol", "Dynamite" 등의 곡은 현대 사회와 문화를 다루며, 이를 비판하고 개선하기 위한 방법을 제안합니다.\n\nBTS는 이러한 다양한 컨셉을 통해 자신의 음악을 통해 다양한 주제와 메시지를 전달하고 있으며, 이는 그들의 음악을 더욱 독특하고 의미 있는 음악으로 만듭니다.'

Create the text generation pipeline with model and tokenizer.

In [8]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Run the pipeline with the custom prompt

In [9]:

messages = [{"role": "user", "content": "BTS 음악의 컨셉은 무엇입니까?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉은 무엇입니까?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS, officially known as Bangtan Sonyeondan,는 K-pop 그룹으로, 한국에서 가장 큰 인기를 끌고 있는 그룹 중 하나입니다. 그들의 음악 컨셉은 다양한 주제를 다루며, 일반적으로는 다음과 같은 주제를 다룹니다:

1. **정서와 감정**: BTS는 많은 곡에서 감정, 정서, 사회적 문제, 개인의 경험 등을 주제로 다루고 있습니다. 예를 들어, "Boy With Luv"는 사랑과 함께하는 것이 중요하다는 메시지를 전달하고 있으며, "DNA"는 자신감과 자존감을 강조합니다.

2. **사회적 문제**: BTS는 다양한 사회적 문제를 다루며, 예를 들어 "Spring Day"는 전쟁과 부모의 죽음을 다루고 있으며, "Mikrokosmos"는 환경 문제와 환경 보호를 주제로 합니다. 또한, "No More Dream"는 사회적 불평등과 권리 struggle을 다루고 있습니다.

3. **개인 성장과 자아 발전**: BTS는 개인의 성장과 자아 발전을 주제로 한 많은 곡을 내보냈습니다. 예를 들어, "Euphoria"는 자신감과 자존감을 강조하며, "Not Today"는 자신의 목표를 향한 노력을 다루고 있습니다.

4. **문화와 민족**: BTS는 한국 문화와 민족의 중요성을 

In [10]:
messages2 = [{"role": "user", "content": "BTS 음악의 컨셉을 3개만 적어주세요."}]

prompt2 = tokenizer.apply_chat_template(
    messages2, tokenize=False, add_generation_prompt=True
)

outputs2 = pipe(prompt2, max_new_tokens=1024, eos_token_id=terminators, do_sample=True)

print(outputs2[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 16 Dec 2024

You are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

BTS 음악의 컨셉을 3개만 적어주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

BTS(방탄소년단)의 음악은 다양한 컨셉과 주제를 다루고 있지만, 몇 가지 주요한 컨셉을 정리해 보겠습니다:

1. **자아 발전과 성장**: BTS의 많은 곡들은 개인의 성장과 발전을 주제로 다룹니다. 예를 들어, "DNA", "Mic Drop", "Boy With Luv" 등은 자신의 능력을 발휘하고, "Euphoria"는 자신을 향한 사랑과 인정을 표현하는 것입니다. 이 컨셉은 개인의 성장과 발전을 주제로 한 음악으로, 인생의 다양한 단계를 통해 성장하는 과정을 그린 것입니다.

2. **사랑과 인연**: BTS는 사랑과 인연을 주제로 많은 곡을 만들어왔습니다. "Spring Day", "Serendipity", "Love Yourself" 등은 사랑과 인연을 다루며, "Not Today"와 "Euphoria"는 사랑과 인정을 표현하는 데 초점을 맞춥니다. 이 컨셉은 사랑과 인연이 인생의 중요한 부분임을 강조합니다.

3. **사회적 문제와 사회적 인식**: BTS는 사회적 문제를 주제로 많은 곡을 만들어왔습니다. "Spring Day"는 한국 전쟁의 추억과 그에 따른 사회적 문제를 다루며, "Not Today"는 군대에 복무하는 soldier들의 삶과 사회적 문제를 주제로 합니다. 또한, "E